In [4]:
import pandas as pd 
import os 
from sqlalchemy import create_engine
import logging
import time

logging.basicConfig(
   filename="logs/ingestion_db.log",
    level=logging.DEBUG,
    format="%(asctime)s-%(levelname)s-%(message)s",
    filemode="a"
    
)

engine = create_engine('sqlite:///inventory.db')

def ingest_db(df,table_name,engine):
    '''this function will ingest the dataframe into database table'''
    df.to_sql(table_name,con=engine, if_exists = 'replace' , index = False)

def load_raw_data():
    '''this function load the csv as a dataframe and ingestion into db '''
    start = time.time()
    for file in os.listdir(r"C:\PROJECT_2025\powerbi2025\Vendor Performance Data Analytics\dataset\data\data"):
         if '.csv' in file:
           df = pd.read_csv(r'C:\PROJECT_2025\powerbi2025\Vendor Performance Data Analytics\dataset\data\data/'+file)
           print(df.shape)
           logging.info(f'ingesting{file} in db')
           ingest_db(df,file[:-4],engine)
    end = time.time()
    total_time = (end-start)/60
    
    logging.info('-----------------------ingestion complete-------------------------------')
    logging.info('ingestion complete')
    logging.info(f'Total time taken :{total_time} minutes')

if __name__ == "__main__":
    load_raw_data()

(206529, 9)
(224489, 9)
(2372474, 16)
(12261, 9)
(12825363, 14)
(5543, 10)


In [10]:
import pandas as pd
import sqlite3
import logging
from ingestion_db import ingest_db

logging.basicConfig(
    filename="logs/ingestion_db.log",
    level=logging.DEBUG,
    format="%(asctime)s-%(levelname)s-%(message)s",
    filemode="a"
)

def create_vendor_summary(conn):
    """This function merges different tables to get the overall vendor summary."""
    vendor_sales_summary = pd.read_sql_query("""
    WITH freight_summary AS (
        SELECT 
            VendorNumber,
            SUM(freight) AS freight_cost
        FROM vendor_invoice
        GROUP BY VendorNumber
    ),

    purchase_summary AS (
        SELECT 
            p.VendorNumber,
            p.VendorName,
            p.Brand,
            p.Description,
            SUM(p.Quantity) AS total_purchase_quantity,
            SUM(p.Dollars) AS total_purchase_dollars,
            pp.PurchasePrice,
            pp.volume,
            pp.price AS actual_price
        FROM purchases AS p
        JOIN purchase_prices AS pp 
            ON p.Brand = pp.Brand
        GROUP BY p.VendorNumber, p.VendorName, p.Brand
    ),

    sales_summary AS (
        SELECT 
            VendorNo,
            Brand,
            SUM(salesDollars) AS total_sales_dollars,
            SUM(SalesPrice) AS total_sale_price,
            SUM(SalesQuantity) AS total_sales_quantity,
            SUM(ExciseTax) AS total_excise_tax
        FROM sales
        GROUP BY VendorNo, Brand
    )

    SELECT 
        ps.VendorNumber,
        ps.VendorName,
        ps.Brand,
        ps.Description,
        ps.total_purchase_quantity,
        ps.total_purchase_dollars,
        ps.PurchasePrice,
        ps.volume,
        ps.actual_price,
        ss.total_sales_dollars,
        ss.total_sale_price,
        ss.total_sales_quantity,
        ss.total_excise_tax,
        fs.freight_cost
    FROM purchase_summary ps
    LEFT JOIN sales_summary ss 
        ON ps.VendorNumber = ss.VendorNo AND ps.Brand = ss.Brand
    LEFT JOIN freight_summary fs 
        ON ps.VendorNumber = fs.VendorNumber
    ORDER BY ps.total_purchase_dollars DESC;
    """, conn)
    return vendor_sales_summary

def clean_data(df):
    """This function cleans the data and adds new columns for analysis."""
    # Ensure required columns exist
    required_columns = ['volume', 'VendorName', 'Description', 'total_sales_dollars', 
                       'total_purchase_dollars', 'total_sales_quantity', 'total_purchase_quantity']
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        logging.error(f"Missing required columns: {missing_columns}")
        raise ValueError(f"Missing required columns: {missing_columns}")

    # Change datatype to float
    df['volume'] = df['volume'].astype('float64')

    # Fill missing values with 0
    df.fillna(0, inplace=True)

    # Remove white spaces from categorical columns
    df['VendorName'] = df['VendorName'].str.strip()
    df['Description'] = df['Description'].str.strip()

    # Create new columns
    df['GrossProfit'] = df['total_sales_dollars'] - df['total_purchase_dollars']
    
    # Calculate ProfitMargin, avoiding division by zero
    df['ProfitMargin'] = (df['GrossProfit'] / df['total_sales_dollars'] * 100).where(
        df['total_sales_dollars'] != 0, 0)
    
    # Calculate StockTurnover, avoiding division by zero
    df['StockTurnover'] = (df['total_sales_quantity'] / df['total_purchase_quantity']).where(
        df['total_purchase_quantity'] != 0, 0)
    
    # Calculate SalesPurchaseRatio, avoiding division by zero
    df['SalesPurchaseRatio'] = (df['total_sales_dollars'] / df['total_purchase_dollars']).where(
        df['total_purchase_dollars'] != 0, 0)

    return df

if __name__ == '__main__':
    # Create database connection
    conn = sqlite3.connect('inventory.db')

    logging.info('Creating vendor summary table')
    summary_df = create_vendor_summary(conn)
    logging.info(summary_df.head())

    logging.info('Cleaning the data...')
    clean_df = clean_data(summary_df)
    logging.info(clean_df.head())

    logging.info('Ingesting data...')
    ingest_db(clean_df, 'vendor_sales_summary', conn)
    logging.info('Ingestion complete')

    # Close the connection
    conn.close()

In [9]:
import pandas as pd 
import sqlite3 
import logging
from ingestion_db import ingest_db

logging.basicConfig(
   filename="logs/ingestion_db.log",
    level=logging.DEBUG,
    format="%(asctime)s-%(levelname)s-%(message)s",
    filemode="a"
    
)


def create_vendor_summary(conn):
    """this function will merge the differnet tables to get the overall vendor summary and adding new columns in the resulatant data"""
    vendor_sales_summary = pd.read_sql_query("""WITH freight_summary AS (
    SELECT 
        VendorNumber,
        SUM(freight) AS freight_cost
    FROM vendor_invoice
    GROUP BY VendorNumber
    ),

    purchase_summary AS (
    SELECT 
        p.VendorNumber,
        p.VendorName,
        p.Brand,
        p.Description,
        SUM(p.Quantity) AS total_purchase_quantity,
        SUM(p.Dollars) AS total_purchase_dollars,
        pp.PurchasePrice,
        pp.volume,
        pp.price AS actual_price
    FROM purchases AS p
    JOIN purchase_prices AS pp 
        ON p.Brand = pp.Brand
    GROUP BY p.VendorNumber, p.VendorName, p.Brand
    ),

    sales_summary AS (
    SELECT 
        VendorNo,
        Brand,
        SUM(salesDollars) AS total_sales_dollars,
        SUM(SalesPrice) AS total_sale_price,
        SUM(SalesQuantity) AS total_sales_quantity,
        SUM(ExciseTax) AS total_excise_tax
    FROM sales
    GROUP BY VendorNo, Brand
    )

    SELECT 
    ps.VendorNumber,
    ps.VendorName,
    ps.Brand,
    ps.Description,
    ps.total_purchase_quantity,
    ps.total_purchase_dollars,
    ps.PurchasePrice,
    ps.volume,
    ps.actual_price,
    ss.total_sales_dollars,
    ss.total_sale_price,
    ss.total_sales_quantity,
    ss.total_excise_tax,
    fs.freight_cost
    FROM purchase_summary ps
    LEFT JOIN sales_summary ss 
        ON ps.VendorNumber = ss.VendorNo AND ps.Brand = ss.Brand
    LEFT JOIN freight_summary fs 
        ON ps.VendorNumber = fs.VendorNumber
    ORDER BY ps.total_purchase_dollars DESC;
    """,conn)
    return vendor_sales_summary

def clean_data(df):
    """this function will clear the data  """
    #changing the datatype to float
    vendor_sales_summary['volume'] = vendor_sales_summary['volume'].astype('float64')

    #filling the missing values with 0
    vendor_sales_summary.fillna(0,inplace=True)

    #removing white spaces from catagorical columns
    vendor_sales_summary['VendorName'] = vendor_sales_summary['VendorName'].str.strip()
    vendor_sales_summary['Description'] = vendor_sales_summary['Description'].str.strip()

    #creating the new columns for better understanding
    vendor_sales_summary['GrossProfit'] = vendor_sales_summary['total_sales_dollars'] -vendor_sales_summary['total_purchase_dollars']
    vendor_sales_summary['ProfitMragin'] = (vendor_sales_summary['GrossProfit']/vendor_sales_summary['total_sales_dollars'])*100
    vendor_sales_summary['StockTurnover'] = vendor_sales_summary['total_sales_quantity']/vendor_sales_summary['total_purchase_quantity']
    vendor_sales_summary['SalesPurchaseRatio'] = vendor_sales_summary['total_sales_dollars'] /vendor_sales_summary['total_purchase_dollars']

    return df

if __name__ == '__main__':
    #creating database connection
    conn = sqlite3.connect('inventory.db')

    logging.info('creating vendor summary table')
    summary_df = create_vendor_summary(conn)
    logging.info(summary_df.head())

    logging.info('cleaning the data...............')
    clean_df = clean_data(summary_df)
    logging.info(clean_df.head())

    logging.info('ingesting data........................')
    ingest_db(clean_df,'vendor_sales_summary',conn)
    logging.info('ingestion complete')

    

In [7]:
#creating database connection
conn = sqlite3.connect('inventory.db')


In [8]:
#checking tables present in database
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type ='table'",conn)
tables

,name
0,begin_inventory
1,end_inventory
2,purchases
3,purchase_prices
4,sales
5,vendor_invoice


In [9]:
pd.read_sql_query("select Count(*) from purchases",conn)

,Count(*)
0,2372474


In [10]:
pd.read_sql_query("select count(*) from sales",conn)

,count(*)
0,12825363


In [11]:
for table in tables['name'] :
    print('-'*50,f'{table}','-'*50)
    print("count of record : ",pd.read_sql(f"select count(*) as count FROM {table} ",conn)['count'].values[0])
    display(pd.read_sql(f"select * from {table} limit 5",conn))

-------------------------------------------------- begin_inventory --------------------------------------------------
count of record :  206529


,InventoryId,Store,City,Brand,Description,Size,onHand,Price,startDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,2024-01-01
1,1_HARDERSFIELD_60,1,HARDERSFIELD,60,Canadian Club 1858 VAP,750mL,7,10.99,2024-01-01
2,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,6,36.99,2024-01-01
3,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,3,38.99,2024-01-01
4,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,6,34.99,2024-01-01


-------------------------------------------------- end_inventory --------------------------------------------------
count of record :  224489


,InventoryId,Store,City,Brand,Description,Size,onHand,Price,endDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,11,12.99,2024-12-31
1,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,7,36.99,2024-12-31
2,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,7,38.99,2024-12-31
3,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,4,34.99,2024-12-31
4,1_HARDERSFIELD_75,1,HARDERSFIELD,75,Three Olives Tomato Vodka,750mL,7,14.99,2024-12-31


-------------------------------------------------- purchases --------------------------------------------------
count of record :  2372474


...

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,58,Gekkeikan Black & Gold Sake,12.99,750mL,750,1,9.28,8320,SHAW ROSS INT L IMP LTD
1,62,Herradura Silver Tequila,36.99,750mL,750,1,28.67,1128,BROWN-FORMAN CORP
2,63,Herradura Reposado Tequila,38.99,750mL,750,1,30.46,1128,BROWN-FORMAN CORP
3,72,No. 3 London Dry Gin,34.99,750mL,750,1,26.11,9165,ULTRA BEVERAGE COMPANY LLP
4,75,Three Olives Tomato Vodka,14.99,750mL,750,1,10.94,7245,PROXIMO SPIRITS INC.


-------------------------------------------------- sales --------------------------------------------------
count of record :  12825363


,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-01,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,2024-01-02,750.0,1,1.57,12546,JIM BEAM BRANDS COMPANY
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-03,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
3,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,2024-01-08,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
4,1_HARDERSFIELD_1005,1,1005,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,2024-01-09,375.0,1,0.79,12546,JIM BEAM BRANDS COMPANY


-------------------------------------------------- vendor_invoice --------------------------------------------------
count of record :  5543


,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,105,ALTAMAR BRANDS LLC,2024-01-04,8124,2023-12-21,2024-02-16,6,214.26,3.47,None
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57,None
2,388,ATLANTIC IMPORTING COMPANY,2024-01-09,8169,2023-12-24,2024-02-16,5,106.60,4.61,None
3,480,BACARDI USA INC,2024-01-12,8106,2023-12-20,2024-02-05,10100,137483.78,2935.20,None
4,516,BANFI PRODUCTS CORP,2024-01-07,8170,2023-12-24,2024-02-12,1935,15527.25,429.20,None


In [12]:
purchases = pd.read_sql_query("select * from purchases where VendorNumber==4466",conn)
purchases

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.40,1
1,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05,1
2,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,6,56.10,1
3,38_GOULCREST_5215,38,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-07,2024-01-19,2024-02-26,9.41,6,56.46,1
4,59_CLAETHORPES_5215,59,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-05,2024-01-19,2024-02-26,9.41,6,56.46,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,81_PEMBROKE_5215,81,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-29,2025-01-04,2025-02-10,9.41,6,56.46,1
2188,62_KILMARNOCK_5255,62,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-28,2025-01-04,2025-02-10,9.35,5,46.75,1
2189,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-28,2025-01-04,2025-02-10,9.41,5,47.05,1
2190,6_GOULCREST_5215,6,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-31,2025-01-04,2025-02-10,9.41,6,56.46,1


In [13]:
purchase_prices = pd.read_sql_query("select * from purchase_prices where VendorNumber==4466",conn)
purchase_prices

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,5215,TGI Fridays Long Island Iced,12.99,1750mL,1750,1,9.41,4466,AMERICAN VINTAGE BEVERAGE
1,5255,TGI Fridays Ultimte Mudslide,12.99,1750mL,1750,1,9.35,4466,AMERICAN VINTAGE BEVERAGE
2,3140,TGI Fridays Orange Dream,14.99,1750mL,1750,1,11.19,4466,AMERICAN VINTAGE BEVERAGE


In [14]:
vendor_invoice = pd.read_sql_query("select * from vendor_invoice where VendorNumber==4466",conn)
vendor_invoice


,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57,None
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-19,8207,2023-12-27,2024-02-26,335,3142.33,16.97,None
2,4466,AMERICAN VINTAGE BEVERAGE,2024-01-18,8307,2024-01-03,2024-02-18,41,383.35,1.99,None
3,4466,AMERICAN VINTAGE BEVERAGE,2024-01-27,8469,2024-01-14,2024-03-11,72,673.20,3.30,None
4,4466,AMERICAN VINTAGE BEVERAGE,2024-02-04,8532,2024-01-19,2024-03-15,79,740.21,3.48,None
5,4466,AMERICAN VINTAGE BEVERAGE,2024-02-09,8604,2024-01-24,2024-03-15,347,3261.37,17.61,None
6,4466,AMERICAN VINTAGE BEVERAGE,2024-02-17,8793,2024-02-05,2024-04-02,72,675.36,3.17,None
7,4466,AMERICAN VINTAGE BEVERAGE,2024-03-01,8892,2024-02-12,2024-03-28,117,1096.05,5.15,None
8,4466,AMERICAN VINTAGE BEVERAGE,2024-03-07,8995,2024-02-19,2024-04-02,129,1209.27,5.44,None
9,4466,AMERICAN VINTAGE BEVERAGE,2024-03-12,9033,2024-02-22,2024-04-16,147,1377.87,6.61,None


In [15]:
purchases.groupby(['Brand','PurchasePrice'])[['Quantity','Dollars']].sum()

,,Quantity,Dollars
Brand,PurchasePrice,,
3140,11.19,4640,51921.60
5215,9.41,4923,46325.43
5255,9.35,6215,58110.25


In [16]:
vendor_invoice.columns

Index(['VendorNumber', 'VendorName', 'InvoiceDate', 'PONumber', 'PODate',
       'PayDate', 'Quantity', 'Dollars', 'Freight', 'Approval'],
      dtype='object')

In [17]:
freight_summary = pd.read_sql_query("select VendorNumber,SUM(freight) as freightcost from vendor_invoice Group by VendorNumber ",conn)
freight_summary

,VendorNumber,freightcost
0,2,27.08
1,54,0.48
2,60,367.52
3,105,62.39
4,200,6.19
...,...,...
121,98450,856.02
122,99166,130.09
123,172662,178.34
124,173357,202.50


In [18]:
purchases.columns

Index(['InventoryId', 'Store', 'Brand', 'Description', 'Size', 'VendorNumber',
       'VendorName', 'PONumber', 'PODate', 'ReceivingDate', 'InvoiceDate',
       'PayDate', 'PurchasePrice', 'Quantity', 'Dollars', 'Classification'],
      dtype='object')

In [19]:
purchase_prices.columns

Index(['Brand', 'Description', 'Price', 'Size', 'Volume', 'Classification',
       'PurchasePrice', 'VendorNumber', 'VendorName'],
      dtype='object')

In [20]:
pd.read_sql(""" select 
                     p.VendorNumber,
                     p.VendorName,
                     p.Brand,
                     pp.PurchasePrice,
                     pp.volume,
                     pp.price as ActualPrice,
                     SUM(p.Quantity) as TotalPurchaseQuantity,
                     SUM(p.Dollars) as TotalPurchaseDollars
                     from purchases as p 
                     join purchase_prices as pp 
                     on p.Brand = pp.Brand
                     group by p.VendorNumber,
                              p.VendorName,
                              p.Brand
                     order by TotalPurchaseDollars""",conn)

,VendorNumber,VendorName,Brand,PurchasePrice,Volume,ActualPrice,TotalPurchaseQuantity,TotalPurchaseDollars
0,2561,EDRINGTON AMERICAS,2166,0.00,750,0.00,2015,0.00
1,7245,PROXIMO SPIRITS INC.,3065,0.71,50,0.99,1,0.71
2,3960,DIAGEO NORTH AMERICA INC,6127,1.47,200,1.99,1,1.47
3,3924,HEAVEN HILL DISTILLERIES,9123,0.74,50,0.99,2,1.48
4,8004,SAZERAC CO INC,5683,0.39,50,0.49,6,2.34
...,...,...,...,...,...,...,...,...
10688,3960,DIAGEO NORTH AMERICA INC,3545,21.89,1750,29.99,138109,3023206.01
10689,3960,DIAGEO NORTH AMERICA INC,4261,16.17,1750,22.99,201682,3261197.94
10690,17035,PERNOD RICARD USA,8068,18.24,1750,24.99,187407,3418303.68
10691,4425,MARTIGNETTI COMPANIES,3405,23.19,1750,28.99,164038,3804041.22


In [21]:
pd.read_sql_query("""select 
                           VendorNo,
                           Brand,
                           SUM(salesDollars) as TotalSalesDollars,
                           SUM(SalesPrice) as TotalSalePrice,
                           SUM(SalesQuantity) as TotalSalesQuantity,
                           SUM(ExciseTax) as TotalExciseTax
                           FROM sales
                           GROUP BY VendorNo , Brand
                           ORDER BY TotalSalesDollars """,conn)

,VendorNo,Brand,TotalSalesDollars,TotalSalePrice,TotalSalesQuantity,TotalExciseTax
0,8004,5287,0.98,0.98,2,0.10
1,9206,2773,0.99,0.99,1,0.05
2,3252,3933,1.98,0.99,2,0.10
3,3924,9123,1.98,0.99,2,0.10
4,10050,3623,1.98,1.98,2,0.10
...,...,...,...,...,...,...
11267,3960,3545,4223107.62,545778.28,135838,249587.83
11268,3960,4261,4475972.88,420050.01,200412,368242.80
11269,17035,8068,4538120.60,461140.15,187140,343854.07
11270,4425,3405,4819073.49,561512.37,160247,294438.66


In [ ]:
# Load the 'sales' table using SQL query
sales = pd.read_sql("SELECT * FROM sales", conn)

In [27]:
query = ("""WITH freight_summary AS (
    SELECT 
        VendorNumber,
        SUM(freight) AS freight_cost
    FROM vendor_invoice
    GROUP BY VendorNumber
),

purchase_summary AS (
    SELECT 
        p.VendorNumber,
        p.VendorName,
        p.Brand,
        p.Description,
        SUM(p.Quantity) AS total_purchase_quantity,
        SUM(p.Dollars) AS total_purchase_dollars,
        pp.PurchasePrice,
        pp.volume,
        pp.price AS actual_price
    FROM purchases AS p
    JOIN purchase_prices AS pp 
        ON p.Brand = pp.Brand
    GROUP BY p.VendorNumber, p.VendorName, p.Brand
),

sales_summary AS (
    SELECT 
        VendorNo,
        Brand,
        SUM(salesDollars) AS total_sales_dollars,
        SUM(SalesPrice) AS total_sale_price,
        SUM(SalesQuantity) AS total_sales_quantity,
        SUM(ExciseTax) AS total_excise_tax
    FROM sales
    GROUP BY VendorNo, Brand
)

SELECT 
    ps.VendorNumber,
    ps.VendorName,
    ps.Brand,
    ps.Description,
    ps.total_purchase_quantity,
    ps.total_purchase_dollars,
    ps.PurchasePrice,
    ps.volume,
    ps.actual_price,
    ss.total_sales_dollars,
    ss.total_sale_price,
    ss.total_sales_quantity,
    ss.total_excise_tax,
    fs.freight_cost
FROM purchase_summary ps
LEFT JOIN sales_summary ss 
    ON ps.VendorNumber = ss.VendorNo AND ps.Brand = ss.Brand
LEFT JOIN freight_summary fs 
    ON ps.VendorNumber = fs.VendorNumber
ORDER BY ps.total_purchase_dollars DESC;
 """)

In [28]:
vendor_sales_summary=pd.read_sql_query(query,conn)

In [39]:
vendor_sales_summary.dtypes

VendorNumber                 int64
VendorName                  object
Brand                        int64
Description                 object
total_purchase_quantity      int64
total_purchase_dollars     float64
PurchasePrice              float64
volume                     float64
actual_price               float64
total_sales_dollars        float64
total_sale_price           float64
total_sales_quantity       float64
total_excise_tax           float64
freight_cost               float64
VendorName                  object
dtype: object

In [30]:
vendor_sales_summary.isnull().sum()

VendorNumber                 0
VendorName                   0
Brand                        0
Description                  0
total_purchase_quantity      0
total_purchase_dollars       0
PurchasePrice                0
volume                       0
actual_price                 0
total_sales_dollars        178
total_sale_price           178
total_sales_quantity       178
total_excise_tax           178
freight_cost                 0
dtype: int64

In [38]:
vendor_sales_summary['VendorName'].unique

<bound method Series.unique of 0        BROWN-FORMAN CORP          
1              MARTIGNETTI COMPANIES
2        PERNOD RICARD USA          
3        DIAGEO NORTH AMERICA INC   
4        DIAGEO NORTH AMERICA INC   
                    ...             
10688    SAZERAC CO INC             
10689    HEAVEN HILL DISTILLERIES   
10690    DIAGEO NORTH AMERICA INC   
10691    PROXIMO SPIRITS INC.       
10692    EDRINGTON AMERICAS         
Name: VendorName, Length: 10693, dtype: object>

In [31]:
vendor_sales_summary['Description'].unique()

array(['Jack Daniels No 7 Black', "Tito's Handmade Vodka",
       'Absolut 80 Proof', ..., 'Concannon Glen Ellen Wh Zin',
       'The Club Strawbry Margarita', 'The Macallan Double Cask 12'],
      shape=(9652,), dtype=object)

In [34]:
vendor_sales_summary['volume'] = vendor_sales_summary['volume'].astype('float64')

In [35]:
vendor_sales_summary.fillna(0,inplace=True)

In [37]:
vendor_sales_summary['VendorName '] = vendor_sales_summary['VendorName'].str.strip()

In [41]:
vendor_sales_summary['GrossProfit'] = vendor_sales_summary['total_sales_dollars'] -vendor_sales_summary['total_purchase_dollars']

In [42]:
vendor_sales_summary

,VendorNumber,VendorName,Brand,Description,total_purchase_quantity,total_purchase_dollars,PurchasePrice,volume,actual_price,total_sales_dollars,total_sale_price,total_sales_quantity,total_excise_tax,freight_cost,VendorName,GrossProfit
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,145080,3811251.60,26.27,1750.0,36.99,5101919.51,672819.31,142049.0,260999.20,68601.68,BROWN-FORMAN CORP,1290667.91
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,164038,3804041.22,23.19,1750.0,28.99,4819073.49,561512.37,160247.0,294438.66,144929.24,MARTIGNETTI COMPANIES,1015032.27
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,187407,3418303.68,18.24,1750.0,24.99,4538120.60,461140.15,187140.0,343854.07,123780.22,PERNOD RICARD USA,1119816.92
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,201682,3261197.94,16.17,1750.0,22.99,4475972.88,420050.01,200412.0,368242.80,257032.07,DIAGEO NORTH AMERICA INC,1214774.94
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,138109,3023206.01,21.89,1750.0,29.99,4223107.62,545778.28,135838.0,249587.83,257032.07,DIAGEO NORTH AMERICA INC,1199901.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10688,8004,SAZERAC CO INC,5683,Dr McGillicuddy's Apple Pie,6,2.34,0.39,50.0,0.49,65.66,1.47,134.0,7.04,50293.62,SAZERAC CO INC,63.32
10689,3924,HEAVEN HILL DISTILLERIES,9123,Deep Eddy Vodka,2,1.48,0.74,50.0,0.99,1.98,0.99,2.0,0.10,14069.87,HEAVEN HILL DISTILLERIES,0.50
10690,3960,DIAGEO NORTH AMERICA INC,6127,The Club Strawbry Margarita,1,1.47,1.47,200.0,1.99,143.28,77.61,72.0,15.12,257032.07,DIAGEO NORTH AMERICA INC,141.81
10691,7245,PROXIMO SPIRITS INC.,3065,Three Olives Grape Vodka,1,0.71,0.71,50.0,0.99,85.14,33.66,86.0,4.46,38994.78,PROXIMO SPIRITS INC.,84.43


In [43]:
vendor_sales_summary['GrossProfit'].min()


np.float64(-52002.780000000006)

In [44]:
vendor_sales_summary['ProfitMragin'] = (vendor_sales_summary['GrossProfit']/vendor_sales_summary['total_sales_dollars'])*100

In [48]:
vendor_sales_summary['StockTurnover'] = vendor_sales_summary['total_sales_quantity']/vendor_sales_summary['total_purchase_quantity']

In [49]:
vendor_sales_summary['SalesPurchaseRatio'] = vendor_sales_summary['total_sales_dollars'] /vendor_sales_summary['total_purchase_dollars']

In [62]:
cursor = conn.cursor()

In [64]:
cursor.execute(""" CREATE TABLE vendor_sales_summary(
        VendorNumber INT,
        VendorName VARCHAR(100),
        Brand INT,
        Description VARCHAR(100),
        total_purchase_quantity INT,
        total_purchase_dollars DECIMAL(15,12),
        PurchasePrice DECIMAL(10,2),
        volume,
        actual_price DECIMAL(10,2),
        total_sales_dollars DECIMAL(15,12),
        total_sale_price DECIMAL(15,12),
        total_sales_quantity INT,
        total_excise_tax DECIMAL(15,12),
        freight_cost DECIMAL(15,12),
        GrossProfit DECIMAL(15,12),
        ProfitMragin DECIMAL(15,12), 
        StockTurnover DECIMAL(15,12),
        SalesPurchaseRatio DECIMAL(15,12),
        PRIMARY KEY (VendorNumber,Brand)
        ); """)

In [69]:
pd.read_sql('select * from vendor_sales_summary',conn)

,VendorNumber,VendorName,Brand,Description,total_purchase_quantity,total_purchase_dollars,PurchasePrice,volume,actual_price,total_sales_dollars,total_sale_price,total_sales_quantity,total_excise_tax,freight_cost,VendorName,GrossProfit,ProfitMragin,StockTurnover,SalesPurchaseRatio
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,145080,3811251.60,26.27,1750.0,36.99,5101919.51,672819.31,142049.0,260999.20,68601.68,BROWN-FORMAN CORP,1290667.91,25.297693,0.979108,1.338647
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,164038,3804041.22,23.19,1750.0,28.99,4819073.49,561512.37,160247.0,294438.66,144929.24,MARTIGNETTI COMPANIES,1015032.27,21.062810,0.976890,1.266830
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,187407,3418303.68,18.24,1750.0,24.99,4538120.60,461140.15,187140.0,343854.07,123780.22,PERNOD RICARD USA,1119816.92,24.675786,0.998575,1.327594
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,201682,3261197.94,16.17,1750.0,22.99,4475972.88,420050.01,200412.0,368242.80,257032.07,DIAGEO NORTH AMERICA INC,1214774.94,27.139908,0.993703,1.372493
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,138109,3023206.01,21.89,1750.0,29.99,4223107.62,545778.28,135838.0,249587.83,257032.07,DIAGEO NORTH AMERICA INC,1199901.61,28.412764,0.983556,1.396897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10688,8004,SAZERAC CO INC,5683,Dr McGillicuddy's Apple Pie,6,2.34,0.39,50.0,0.49,65.66,1.47,134.0,7.04,50293.62,SAZERAC CO INC,63.32,96.436186,22.333333,28.059829
10689,3924,HEAVEN HILL DISTILLERIES,9123,Deep Eddy Vodka,2,1.48,0.74,50.0,0.99,1.98,0.99,2.0,0.10,14069.87,HEAVEN HILL DISTILLERIES,0.50,25.252525,1.000000,1.337838
10690,3960,DIAGEO NORTH AMERICA INC,6127,The Club Strawbry Margarita,1,1.47,1.47,200.0,1.99,143.28,77.61,72.0,15.12,257032.07,DIAGEO NORTH AMERICA INC,141.81,98.974037,72.000000,97.469388
10691,7245,PROXIMO SPIRITS INC.,3065,Three Olives Grape Vodka,1,0.71,0.71,50.0,0.99,85.14,33.66,86.0,4.46,38994.78,PROXIMO SPIRITS INC.,84.43,99.166079,86.000000,119.915493


In [68]:
  vendor_sales_summary.to_sql('vendor_sales_summary',conn,if_exists='replace',index= False)

10693